EE Genie as an interactive Earth Engine GenAI assistant that works
with geemap in Colab and can retrieve and analyze images.
<a target='_blank' href='https://colab.research.google.com/github/google/earthengine-community/blob/master/experimental/ee_genie.ipynb'>   <img src='https://colab.research.google.com/assets/colab-badge.svg' alt='Open In Colab'/> </a>


Author: Simon Ilyushchenko (simonf@google.com)

Last update: 2025-03-31

**USING THIS AGENT IS UNSAFE**. It directly runs LLM-produced code, and thus
should only be used for demonstration purposes. However, Colab serves as
a moderately effective sandbox - the damage would be limited to whatever
this notebook has access to.

**Description**

The EE Genie notebook is shown as a two-column view.
The left column is the chat history showing all interactions with LLM. The right column contains an interactive instance of geemap.

The default prompt at the very bottom is "show a whole continent Australia DEM visualization using a palette that captures the elevation range". Just hit enter to accept it and wait for the output to appear. This works like a regular LLM chat, so you can continue talking about your map - e.g., you can say "Now scroll the map up to India and verify the image is correct".

Some other queries that work most of the time are commented out in the code where `command_input` is defined.

The agent fetches the same tiles that are loaded on geemap, then stitches them together into one large image and sends it to a model for textual description.

**Annoyance**

Due to problems with Javascript/Python interaction, the agent has to stop running after it moves or pans geemap. When this happens, the agent icon will change to 🙏. Just hit enter in the chat box when you see this to continue analysis.

**Installation**

To use it, you need two things:
1. Earth Engine access
2. Google API key to use with the genai client.

You need a Google Cloud Project to associate your requests with. [Use these instructions](https://developers.google.com/earth-engine/cloud/earthengine_cloud_project_setup) and set EE_PROJECT_ID notebook secret to your project id.

Next you need to get an Generative AI API key [here](https://aistudio.google.com/app/prompts/new_chat).

To save this key in the notebook, click on the key icon in Colab on the left-hand side and add your key as a secret with the name GOOGLE_API_KEY. Make sure the value has no newlines.

To use EE Genie, run the cells, then scroll to the UI at the bottom.
Change the task that you want the agent to work on and hit enter.

# Imports

In [ ]:
!pip install --quiet geedim tenacity

In [ ]:
import contextlib

import enum
import io
import json
import re
import math
import os
import sys
import time

import numpy as np
import PIL
import requests

import ee
import geemap
from geemap import temp_file_path
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Javascript

from google.cloud import storage

from google import genai
from google.genai import types

from google.colab import userdata

from tenacity import retry, stop_after_attempt, wait_fixed, retry_if_exception

# Initialization

In [ ]:
ee.Authenticate(scopes=['https://www.googleapis.com/auth/earthengine.readonly'])
ee.Initialize(project=userdata.get('EE_PROJECT_ID'))
storage_client = storage.Client()
bucket = storage_client.get_bucket('earthengine-stac')

# Score to aim for (on the 0-1 scale). The exact meaning of what "score" means
# is left to the LLM.
target_score = 0.8

# Count of analysis rounds
round = 1

Map = geemap.Map()
Map.add("layer_manager")

analysis_model = None
map_dirty = False

#model_name = 'gemini-1.5-pro-latest'
model_name = 'gemini-2.0-flash'
#model_name = 'gemini-2.0-pro-exp-02-05'
#model_name = 'gemini-2.5-pro-exp-03-25'

# UI widget definitions

In [ ]:
# We define the widgets early because some functions will write to the debug
# and/or chat panels.

task = "Show a whole continent Australia DEM visualization using a palette that captures the elevation range"
#task='show NYC'
#task='show an area with many center pivot irrigation circles'
#task='show consequences of a large fire'
#task='show an open pit mine'
#task='a sea port'
#task='flood consequences'
#task='show an interesting modis composite with the relevant visualization and analyze it over Costa Rica'

# Create output area with a unique ID
output_id = f"output_{int(time.time())}"
chat_output = widgets.Output(
    layout=widgets.Layout(
        width='50%',
        height='600px',
        border='1px solid black',
        overflow='auto'
    )
)
# Add a unique CSS class to output area
chat_output.add_class(output_id)

def scroll_to_bottom():
  js_code = f"""
      requestAnimationFrame(() => {{
          const element = document.querySelector('.{output_id}');
          if (element) {{
              element.scrollTop = element.scrollHeight;
          }}
      }});
  """
  display(Javascript(js_code))

def chat_display(text: str) -> None:
  text = text.strip().rstrip('\nNone')
  with chat_output:
    display(HTML(f"<p style='white-space: pre-wrap;'>{text}</p>"))
  scroll_to_bottom()

chat_display(f'LLM: {model_name}\n')

<IPython.core.display.Javascript object>

# Simple functions that LLM will call

In [ ]:
def set_center(x: float, y: float, zoom: int) -> str:
  """Sets the map center to the given coordinates and zoom level and
  returns instructions on what to do next."""
  chat_display(f"SET_CENTER({x}, {y}, {zoom})\n")
  Map.set_center(x, y)
  Map.zoom = zoom
  global map_dirty
  map_dirty = True
  return (
    'Do not call any more functions in this request to let geemap bounds '
    'update. Wait for user input.')

def get_dataset_description(dataset_id: str) -> str:
  """Fetches JSON STAC description for the given Earth Engine dataset id."""
  chat_display(f'LOOKING UP {dataset_id}\n')
  parent = dataset_id.split('/')[0]

  # Get the blob (file)
  path = os.path.join('catalog', parent, dataset_id.replace('/', '_')) + '.json'
  blob = bucket.blob(path)

  if not blob.exists():
    return 'dataset file not found: ' + path

  file_contents = blob.download_as_string().decode()
  return file_contents

def show_layer(python_code: str) -> str:
    """Execute the given Earth Engine Python client code and add the result to
    the map. Returns the status message (success or error message)."""
    Map.layers = Map.layers[:2]
    while '\\"' in python_code:
      python_code = python_code.replace('\\"', '"')
    chat_display(f'IMAGE:\n {python_code}\n')
    try:
      locals = {}
      exec(f"import ee; im = {python_code}", {}, locals)
      Map.addLayer(locals['im'])
    except Exception as e:
      chat_display(f"ERROR: {e}"  )
      return str(e)
    return 'success'

def inner_monologue(thoughts: str) -> str:
  """Sends the current thinking of the LLM model to the user so that they are
  aware of what the model is thinking between function calls."""
  chat_display(f'THOUGHTS:\n {thoughts}\n')
  return 'success'

# Functions for textual analysis of images

In [ ]:
def _lat_lon_to_tile(lon, lat, zoom_level):
  """Converts latitude and longitude to Mercator coordinates."""
  x_merc = (lon + 180) / 360
  y_merc = (
      1
      - math.log(
          math.tan(math.radians(lat)) + 1 / math.cos(math.radians(lat))
      )
      / math.pi
  ) / 2

  # Calculate number of tiles
  n = 2**zoom_level

  # Convert to tile coordinates
  X = int(x_merc * n)
  Y = int(y_merc * n)

  return X, Y

def analyze_image(additional_instructions:str) -> str:
  """Returns GenAI image analysis describing the current map image.
  Optional additional instructions might be passed to target the analysis
  more precisely.
  """
  global map_dirty
  if map_dirty:
      print('MAP DIRTY')
      return 'Map is not ready. Stop further processing and ask for user input'

  try:
    return _analyze_image(additional_instructions)
  except ValueError as e:
    return str(e)

def _analyze_image(additional_instructions:str) -> str:
  if not additional_instructions:
    additional_instructions = ''
  layers = list(Map.ee_layer_dict.values())
  if not layers:
      return "No data layers loaded"
  image_temp_file = temp_file_path(extension="jpg")
  # Read the tile from the topmost (-1) layer.
  layer_name = layers[-1]["ee_layer"].name
  Map.layer_to_image(layer_name, output=image_temp_file, scale=Map.get_scale())
  image = PIL.Image.open(image_temp_file)

  image_array = np.array(image)
  image_min = np.min(image_array)
  image_max = np.max(image_array)

  # Skip an LLM call when we can simply tell that something is wrong.
  # (Also, LLMs might hallucinate on uniform images.)
  if image_min == image_max:
    return (
        f'The image tile has a single uniform color with value '
        f'{image_min}.'
    )

  query = """You are an objective, precise overhead imagery analyst.
Describe what the provided map tile depicts in terms of:

1. The colors, textures, and patterns visible in the image.
2. The spatial distribution, shape, and extent of distinct features or regions.
3. Any notable contrasts, boundaries, or gradients between different areas.

Avoid making assumptions about the specific geographic location, time period,
or cause of the observed features. Focus solely on the literal contents of the
image itself. Clearly indicate which features look natural, which look human-made,
and which look like image artifacts. (Eg, a completely straight blue line
is unlikely to be a river.)

If the image is ambiguous or unclear, state so directly. Do not speculate or
hypothesize beyond what is directly visible.

If the image is of mostly the same color (white, gray, or black) with little
contrast, just report that and do not describe the features.

Use clear, concise language. Avoid subjective interpretations or analogies.
Organize your response into structured paragraphs.
"""
  if additional_instructions:
    query += additional_instructions
  client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))
  image_response = client.models.generate_content(
      model=model_name,
      contents=[query, image]
  )
  try:
    chat_display(f'ANALYSIS RESULT: {image_response.text}\n')
    return image_response.text
  except ValueError as e:
    chat_display(f'UNEXPECTED IMAGE RESPONSE: {e}')
    chat_display(image_response)


# Function for scoring how well image analysis corresponds to the user query.

In [ ]:
# Note that we ask for the score outside of the main agent chat to keep
# the scoring more objective.

scoring_system_prompt = """
After looking at the user query and the map tile analysis, start
your answer with a number between 0 and 1 indicating how relevant
the image is as an answer to the query. (0=irrelevant, 1=perfect answer)

Make sure you have enough justification to definitively declare the analysis
relevant - it's better to give a false negative than a false positive. However,
the image analysis identifies specific matching landmarks (eg, the
the outlines of Manhattan island for a request to show NYC), believe it.

Do not assume  too much (eg, that the presence of green doesn't by itself mean the
image shows forest); attempt to find multiple (at least three) independent
lines of evidence before declaring victory and cite all these lines of evidence
in your response.

Be very, very skeptical - look for specific features that match only the query
and nothing else (eg, if the query looks for a river, a completely straight blue
line is unlikely to be a river). Think about what size the features are based on
the zoom level and whether this size matches the feature size expected from
first principles.

If there is ambiguity or uncertainty, express it in your analysis and
lower the score accordingly. If the image analysis is inconclusive, try zooming
out to make sure you are looking at the right spot. Do not reduce the score if
the analysis does not mention visualization parameters - they are just given for
your reference. The image might show an area slightly larger than requested -
this is okay, do not reduce the score on this account.
"""

def score_response(query: str, visualization_parameters: str, analysis: str) -> str:
  """Returns how well the given analysis describes a map tile returned for
  the given query. The analysis starts with a number between 0 and 1.

  Arguments:
    query: user-specified query
    visualization_parameters: description of the bands used and visualization
      parameters applied to the map tile
    analysis: the textual description of the map tile
  """
  chat_display(f"VIZ PARAMS: {visualization_parameters}\n")
  question = (
      f"""For user query {query} please score the following analysis:
      {analysis}. The answer must start with a number between 0 and 1.""")
  if visualization_parameters:
    question += (
        f"""Do not assume that common bands or visualization
        parameters should have been used, as the visualization used the
        following parameters: {visualization_parameters}""")

  result = analysis_model.ask(question)
  global round
  chat_display(f'SCORE #{round}:\n {result}\n')
  round += 1
  return result

# Main prompt for the agent

In [ ]:
system_prompt = f"""
The client is running in a Python notebook with a geemap Map displayed.
When composing Python code, do not use getMapId - just return the single-line
layer definition like 'ee.Image("USGS/SRTMGL1_003")' that we will pass to
Map.addLayer(). Do not escape quotation marks in Python code.

Be sure to use Python, not Javascript, syntax for keyword parameters in
Python code (that is, "function(arg=value)") Using the provided functions,
respond to the user command following below (or respond why it's not possible).
If you get an Earth Engine error, attempt to fix it and then try again.

Before you choose a dataset, think about what kind of dataset would be most
suitable for the query. Also think about what zoom level would be suitable for
the query, keeping in mind that for high-resolution image collections higher
zoom levels are better to speed up tile loading.

Once you have chosen a dataset, read its description using the provided function
to see what spatial and temporal range it covers, what bands it has, as well as
to find the recommended visualization parameters. Explain using the inner
monlogue function why you chose a specific dataset, zoom level and map location.

Prefer mosaicing image collections using the mosaic() function, don't get
individual images from collections via
'first()'. Choose a tile size and zoom level that will ensure the
tile has enough pixels in it to avoid graininess, but not so many
that processing becomes very expensive. Do not use wide date ranges
with collections that have many images, but remember that Landsat and
Sentinel-2 have revisit period of several days. Do not use sample
locations - try to come up with actual locations that are relevant to
the request.

Use Landsat Collection 2, not Landsat Collection 1 ids. If you are getting
repeated errors when filtering by a time range, read the dataset description
to confirm that the dataset has data for the selected range.

Important: after using the set_center() function, just say that you have called
this function and wait for the user to hit enter, after which you should
continue answering the original request. This will make sure the map is updated
on the client side.

Once the map is updated and the user told you to proceed, call the analyze_image
function() to describe the image for the same location that will be shown in
geemap. If you pass additional instructions to analyze_image(), do not disclose
what the image is supposed to be to discourage hallucinations - you can only tell
the analysis function to pay attention to specific areas (eg, center or top left)
or shapes (eg, a line at the bottom) in the image. You can also tell the analysis
function about the chosen bands, color palette and min/max visualization
parameters, if any, to help it interpret the colors correctly. If the image
turns out to be uniform in color with no features,
use min/max visualization parameters to enhance contrast.

Frequently call the inner_monologue() functions to tell the user about your
current thought process. This is a good time to reflect if you have been running
into repeated errors of the same kind, and if so, to try a different approach.

When you are done, call the score_response() function to evaluate the analysis.
You can also tell the scoring function about the chosen bands, color palette
and min/max visualization parameters, if any. If the analysis score is below
{target_score},
keep trying to find and show a better image. You might have to change the dataset,
map location, zoom level, date range, bands, or other parameters - think about
what went wrong in the previous attempt and make the change that's most likely
to improve the score.
"""

# Class for LLM chat with function calling

In [ ]:
def _is_429_error(exception):
  """Checks if the exception string contains '429'."""
  return '429' in str(exception)


gemini_tools=[
        set_center,
        show_layer,
        analyze_image,
        inner_monologue,
        get_dataset_description,
        score_response
]

class Gemini():
  """Gemini LLM."""

  def __init__(self, system_prompt, tools=None):
    if not tools:
      tools = []
    self.system_prompt  = system_prompt
    self.client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))
    self._chat = self.client.chats.create(
        model=model_name,
        config=types.GenerateContentConfig(
            system_instruction=system_prompt,
            temperature=0.1,
            tools=tools
        ),
    )

  @retry(
      stop=stop_after_attempt(5), # Stop after 5 attempts
      wait=wait_fixed(10),       # Wait 10 seconds between attempts
      retry=retry_if_exception(_is_429_error) # Retry only if it's a 429 error
  )
  def ask(self, question, temperature=0):
    client = genai.Client(api_key=userdata.get('GOOGLE_API_KEY'))
    chat = client.chats.create(
        model=model_name,
        config=types.GenerateContentConfig(
            system_instruction=self.system_prompt,
        ),
    )
    return chat.send_message(question).text

  @retry(
      stop=stop_after_attempt(5), # Stop after 5 attempts
      wait=wait_fixed(10),       # Wait 10 seconds between attempts
      retry=retry_if_exception(_is_429_error) # Retry only if it's a 429 error
  )
  def chat(self, question):
    """Sends a single message to the LLM and returns its response."""
    return self._chat.send_message(question).text


model = Gemini(system_prompt, gemini_tools)
analysis_model = Gemini(scoring_system_prompt)

# UI functions

In [ ]:
def set_cursor_waiting():
  js_code = """
  document.querySelector('body').style.cursor = 'wait';
  """
  display(HTML(f"<script>{js_code}</script>"))

def set_cursor_default():
  js_code = """
  document.querySelector('body').style.cursor = 'default';
  """
  display(HTML(f"<script>{js_code}</script>"))

def on_submit(widget):
  global map_dirty
  map_dirty = False
  command_input.description = '❓'
  command = widget.value
  if not command:
    command = 'go on'
  chat_display('> ' + command + '\n')
  widget.value = ''
  set_cursor_waiting()
  command_input.description = '🤔'
  response = model.chat(command)
  if map_dirty:
    command_input.description = '🙏'
  else:
    command_input.description = '❓'
  set_cursor_default()
  if response:
    response = response.strip()
  if not response:
    response = '<EMPTY RESPONSE, HIT ENTER>'
  chat_display(response + '\n')
  command_input.value = ''

# UI layout

In [ ]:
table = widgets.HBox([chat_output, Map],  layout=widgets.Layout(width='100%'))
command_input = widgets.Text(
    placeholder='Type your message and press Enter...',
    description='❓',
    value = task,
    layout=widgets.Layout(width='95%')
)
command_input.on_submit(on_submit)

# Container for the UI elements
ui = widgets.VBox([
    table,
    command_input,
], layout=widgets.Layout(width='100%'))

# Add CSS styling
display(HTML("""
<style>
.widget-text input[type="text"] {
    width: 100% !important;
    padding: 8px;
    margin: 8px 0;
    box-sizing: border-box;
}
.jupyter-widgets-output-area {
    overflow-y: auto !important;
}
</style>
"""))

# Display the layout
display(ui)
print('❓ = waiting for user input')
print('🙏 = waiting for user to hit enter after calling set_center()')
print('🤔 = thinking')
print('💤 = sleeping due to retries')
print('🆁 = Gemini recitation error')